In [1]:
import math
import json
import requests
import itertools
import numpy as np
import time
import praw
from datetime import datetime, timedelta

In [3]:

def make_request(uri, max_retries = 5):
    current_try = 1
    def fire_away(uri,current_try):
        if current_try < max_retries:
            try:
                response = requests.get(uri)
                if response.status_code == 200:
                    return json.loads(response.content)
            except:
                current_try+=1
                time.sleep(1)
                fire_away(uri,current_try)
        else:
            print('Cannot establish connection or wrong uri',uri)
            return None
    return fire_away(uri,current_try)

#make_request('https://api.pushshift.ioo/reddit/search/submission/?q=science&size=2&fields=selftext,title,author,url,id,full_link')    

In [17]:

def run_per_day(subreddit,start,end):
    uri = 'https://api.pushshift.io/reddit/search/submission/?subreddit={}&size=2&fields=selftext,title,author,url,id,full_link&before={}&after{}'
    uri=uri.format(subreddit,start,end)
    #print(uri)
    response=make_request(uri)
    print(response)
    # Collect comments for each submission
    # Save to file
    # id w44W3IfIXbrE8w
    # key j0el6C-wFuaSrMlPeA94aCXRWVQ

    
run_per_day('news',1550596542,1550682943)


{'data': [{'author': 'QuirkySpiceBush', 'full_link': 'https://www.reddit.com/r/news/comments/ascqma/israel_shoots_for_the_moon_with_privately_funded/', 'id': 'ascqma', 'selftext': '', 'title': 'Israel shoots for the moon with privately funded spacecraft', 'url': 'https://www.cnn.com/2019/02/18/middleeast/israel-spacecraft-beresheet-launch-intl/index.html'}, {'author': 'csaabc', 'full_link': 'https://www.reddit.com/r/news/comments/ascqe4/csa_course/', 'id': 'ascqe4', 'selftext': '', 'title': 'CSA Course', 'url': 'https://www.classifiedads.com/training_education/f9f9wb6dw1898'}]}


In [12]:
reddit = praw.Reddit(client_id='w44W3IfIXbrE8w',
                     client_secret='j0el6C-wFuaSrMlPeA94aCXRWVQ',
                     redirect_uri='http://localhost:8080',
                     user_agent='testscript by UR')
print(reddit.auth.url(['identity'], '...', 'permanent'))
    

https://www.reddit.com/api/v1/authorize?client_id=w44W3IfIXbrE8w&duration=permanent&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&scope=identity&state=...


In [16]:
submission = reddit.submission(id='atbfyz')
submission.comments.replace_more(limit=None)
for top_level_comment in submission.comments:
    print("************Top level comment*****")
    print(top_level_comment.body)
    print("-----------------REPLIES--------------------------")
    for second_level_comment in top_level_comment.replies:
        print(second_level_comment.body)
    print('-------------END----------------')

************Top level comment*****
Air, because falling to your death is avoidable. 

Edit: I ride a specific type of bison. 
-----------------REPLIES--------------------------
You could also stay on the ground. 
Air, because you could fucking fly! how has this not been said?!
With enough creativity, all elements can avoid falling to their death 
with air you can basically control all the other elements
That was my first thought too! I am terrified of heights. If I was an airbender my worst phobia will be gone in the wind... 
Well if you are falling into earth several earthbenders through the series have sunk themselves in the ground to lessen the fall.
As a person that fears height, I love this idea
What you are saying is... there is no fall damage?
As a skydiver , I like this logic. Don't need a chute if you're able to manipulate the air around you.
Well if you are smart enough then falling to your death is avoidable regardless of being an air bender or not
Falling to death is avoida

Beat me to it
-------------END----------------
************Top level comment*****
I read this as "if you were on a bender"
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
This question has a very different reading in the UK
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Bend my shiny metal ass
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
uhhh fire duhh
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Fire because someone needs to attack all these other nations living in harmony.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
I thought you meant something else at first... 
-----------------REPLIES--------------------------
-----------

************Top level comment*****
air, so I could travel via little tornado like simon belmont 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Maybe air, I would save so much from car expenses!!😭
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Happy cake day 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Earth. Mostly because the earth bending style in the show is based off the style of karate I already practice
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Air so that I can fly
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Cheese 
-----------------REPLIES--------------------------
-------------END----------------
************Top

Fire because they get lightning and combustion and it's the coolest and dragons can teach you to be stronger 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
If water bending includes healing, definitely water.  It might save my life if I'm in an accident.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
I guess I would be whatever element could remove these posts.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Airbending lets me fly
Waterbending is probably the most versatile
Firebending is just destructive and since I dont need it for combat gonna not go with this one.

Earthbending would be easiest to monetize.  A one man construction team to build the foundation of any building could make a lot of money with relatively little effort.

Hmmmmmm

Probably waterbending since I

Earth bending would be the most useful, but I’d love to be a villain about being able to blood bend, so I’d opt for water bending. 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Water water water. I do NOT understand the logic towards anything other to be honest 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Water. I'm a geochemist cleaning up uranium groundwater plumes and I'd be some sort of professional superhero if I could change estimated cleanup times from 500+ years to 2.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Fire. I'm an arson at heart I guess..
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Water. Nothing is stronger. more versatile, or useful.
-----------------REPLIES---------------

...

-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
fire cause i suck at making campfires
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Air. Because I want to fly.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
I wouldn't want it to be air because I serve the drunken public and I might take their breath away. . So water I guess. Wait. Fire? I can't think of a situation where I'm not using my powers for evil. Might as well go back to air. 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
As a Brit I thought this post was asking if I was homosexual. 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Water, so I could was my butt hole out after

-------------END----------------
************Top level comment*****
I don't think I get to choose. Michigan is water bending land, whatever they called it.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
I'd be a spunkbender. If she shuts her legs as im about to nut, i can prevent the nut from going inside her, pull it out and thrust it into her eye. 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Water
I want to heal 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Air, because then I would be able to fly.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Water: stop the polar ice caps from melting and prevent further global warming. 
-----------------REPLIES--------------------------
-------------END------

-------------END----------------
************Top level comment*****
Bender means something different in the UK
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Earth because metal and magma bending seem neat.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
I totally thought fire was the obvious answer because its the only element that isnt manipulated, but is actually created by the bender. But after reading all the answers I dont know! 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Water, never be thirsty again, I could drown people who annoy me, and once I become skilled enough I could bend the blood in people to make them do what I want or to drain their blood. With practice I’d also be able to bend water out of air to make an area really dry. 
-----------------REPLIES----

Air, you could hypothetically fly with air bending. Also there is a lot of shit you could get away with if you were an Airbender.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Air anyday, all you need is a kite thingy to fly. No more traffic plus air is almost everywhere.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
I would bend what I want, when I want, who I want.
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Blackjack and hookets
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
I am bender please insert liquor! 
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Earth, because I could take rocks and then smooth them out and make fu

-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
Water, a lot of medical applications aside healing. Blood transfusion, spinal fluid bending, restablish microcirculation on severed limbs, hemorrhage control, cryotherapy...

Just imagining it give me chills. See what I did there?
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
You dont even KNOW how much i wanted to just great little flames on my palms, so ofc I'd take moflippin Fire Bending! I'M AN ETERNAL FLAME, BABY
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
I would chose air because I can fly anywhere then and I could through anything with air basically
-----------------REPLIES--------------------------
-------------END----------------
************Top level comment*****
earth because fortnite irl
-----------------REPLIE